In [2]:
import numpy
import math
from parse import parse
from IPython.display import Math, Latex, display, display_latex

In [21]:
""" CR15 graph library """
class Graph(object):

    def __init__(self, graph_dict={}):
        """ initializes a graph object """
        self.__graph_dict = graph_dict.copy()

    def vertices(self):
        """ returns the vertices of a graph """
        return list(self.__graph_dict.keys())

    def edges(self):
        """ returns the edges of a graph """
        return self.__generate_edges()
    
    def connected_components(self):
        return self.__generate_components()

    def add_vertex(self, vertex):
        """ If vertex is not in self.__graph_dict, a key "vertex" with an empty
        list as a value is added to the dictionary. Otherwise nothing has to be 
        done. To complete."""
        if not vertex in self.__graph_dict:
            self.__graph_dict[vertex] = []
        

    def add_edge(self, edge):
        """ assumes that edge is of type set, tuple or list. No loops or 
        multiple edges. To complete."""
        my_edge = list(edge)
        if len(my_edge) != 2: raise WrongSizeForEdge()
        u = edge.pop()
        v = edge.pop()
        if u in self.__graph_dict and v in self.__graph_dict:
            if u != v:
                if v not in self.__graph_dict[u]:
                    self.__graph_dict[u].append(v)
                if u not in self.__graph_dict[v]:
                    self.__graph_dict[v].append(u)
        else:
            raise VerticesNotDecleared()
            

    def __generate_edges(self):
        """ A static method generating the edges of the graph "graph". Edges 
        are represented as sets two vertices, with no loops. To complete."""
        edges = []
        for v, edges_list in self.__graph_dict.items():
            for u in edges_list:
                if v < u:
                    edges.append(set([v,u]))
        return edges
    
    def vertex_degrees(self):
        """Return a dictionary degree"""
        degrees = {}
        for v, edges_list in self.__graph_dict.items():
            degrees[v] = len(edges_list)
        return degrees
    
    def vertex_degree(self, vertex):
        """Return a dictionary degree"""
        return len(self.__graph_dict[vertex])
    
    def find_isolated_vertices(self):
        zero_set = set()
        for v, edges_list in self.__graph_dict.items():
            if len(edges_list) == 0:
                zero_set.add(v)
                
    def density(self):
        deg = self.vertex_degrees()
        density = 0
        for v, d in deg.items():
            density += d
        density /=  (len(deg) -1) *len(deg)
        return density
                    
    def dict(self):
        return self.__graph_dict
    
    def degree_sequence(self):
        deg = self.vertex_degrees()
        deg_list = [v for v in deg.values()]
        deg_list.sort(reverse=True)
        return tuple(deg_list)
    
    @staticmethod
    def erdos_gallai(deg_seq):
        #deg_seq = self.degree_sequence()
        even_number = 0
        for v in deg_seq:
            even_number += v
        if even_number % 2 == 1 :
            return False
        sumOfdi = 0
        for k in range(len(deg_seq)):
            sumOfdi += deg_seq[k]
            sumOfMin = k*(k+1)
            for i in range(k, len(deg_seq)):
                sumOfMin += min(deg_seq[i], k+1)
            if sumOfMin < sumOfdi:
                return False
        return True
    
    def global_clustering_coefficient(self):
        triangle = 0
        triplet = 0
        for v in self.__graph_dict:
            for u in self.__graph_dict[v]:
                for w in self.__graph_dict[v]:
                    if u != w: 
                        triplet += 1
                    if u != w and w in self.__graph_dict[u]:
                        triangle += 1
        return triangle / triplet
    
    """ Graph traversal """
    def components_BFS(self, vertices, comps, base_vertice):
        if not vertices:
            return
        new_vertices = []
        for v in vertices:
            comps[v] = base_vertice
            for u in self.__graph_dict[v]:
                if comps[u] == u and u != base_vertice:
                    new_vertices.append(u)
        self.components_BFS(new_vertices, comps, base_vertice)
    
    def __generate_components(self):
        comps = {}
        for u in self.__graph_dict:
            comps[u] = u
        for u in self.__graph_dict:
            if comps[u] == u:
                self.components_BFS([u], comps, u)
        return comps
    
    def shortest_path_BFS(self, vertices, seen, d, goal):
        if not vertices:
            return math.inf
        if goal in vertices:
            return d
        new_vertices = set()
        for v in vertices:
            seen.add(v)
        for v in vertices:
            for u in self.__graph_dict[v]:
                if u not in seen:
                    new_vertices.add(u)
        return self.shortest_path_BFS(new_vertices, seen, d+1, goal)
    
    def shortest_path(self, s, t):
        return self.shortest_path_BFS({s}, set(), 0, t)
    
    def diameter(self):
        diam = 0
        for u in self.__graph_dict:
            for v in self.__graph_dict:
                if u<v:
                    l = self.shortest_path(u,v)
                    if l>diam:
                        diam = l
        return diam
    
    def diameter_component(self, u):
        """ Return the diameter of the component containing vertex u """
        # First get the component of u
        component = set()
        comps = self.connected_components()
        for v in self.__graph_dict:
            if comps[u] == comps[v]:
                component.add(v)
        diam = 0
        for v in component:
            for w in component:
                if v<w:
                    l = self.shortest_path(v,w)
                    if l>diam:
                        diam = l
        return diam
    
    def biggest_component_diameter(self):
        if not self.vertices(): return 0
        
        # First determine the biggest component #
        comps = self.connected_components()
        comps_size = {}
        for u in comps.values():
            comps_size[u] = 0
        for v,u in comps.items():
            comps_size[u] += 1
        biggest = tuple(comps.values())[0]
        max_size = comps_size[biggest]
        for u in comps.values():
            if comps_size[u] > max_size:
                max_size = comps_size[u]
                biggest = u
        
        # Return the diameter of the corresponding component
        return self.diameter_component(biggest)
        
        
    """ Static methods for defining classical graphs """
    @staticmethod
    def clique(n):
        d = {}
        s = set(i+1 for i in range(n))
        for i in range(n):
            d[i+1] = s.difference(set({i+1}))
        return Graph(d)
    
    @staticmethod
    def no_edges(n):
        d = {}
        for i in range(n):
            d[i+1] = []
        return Graph(d)
    
    """ Importing from a text file """
    @staticmethod
    def from_txt(file):
        G = Graph()
        lines = open(file).readlines()
        for l in lines:
            p = parse('{:d}\t{:d}', l)
            G.add_vertex(p[0])
            G.add_vertex(p[1])
            G.add_edge({p[0],p[1]})
        return G

In [22]:
G = {
      "a": ["c", "d", "g"],
      "b": ["c", "f"],
      "c": ["a", "b", "d", "f"],
      "d": ["a", "c", "e", "g"],
      "e": ["d"],
      "f": ["b", "c"],
      "g": ["a", "d"]
    }
graph = Graph(G)
print("Vertices of graph:")
print(graph.vertices())
print("Edges of graph:")
print(graph.edges())
print("Degrees of the graph:")
print(graph.vertex_degrees())
print(graph.vertex_degree("d"))
print("Isolated vertices:")
print(graph.find_isolated_vertices())
print("Graph density:")
print(graph.density())

graph_empty = Graph.no_edges(5)
graph_complete = Graph.clique(5)

print(graph_empty.density())
print(graph_complete.density())

print("Degree sequence:")
print(graph.degree_sequence())
print("Erdos GAllai:")
print(graph.erdos_gallai(graph.degree_sequence()))
print(graph.erdos_gallai(tuple([1])))
print("Clustering:")
print(graph.global_clustering_coefficient())
print(graph_complete.global_clustering_coefficient())

Vertices of graph:
['c', 'f', 'd', 'a', 'b', 'e', 'g']
Edges of graph:
[{'c', 'd'}, {'c', 'f'}, {'d', 'e'}, {'d', 'g'}, {'c', 'a'}, {'d', 'a'}, {'a', 'g'}, {'c', 'b'}, {'f', 'b'}]
Degrees of the graph:
{'c': 4, 'f': 2, 'd': 4, 'a': 3, 'b': 2, 'g': 2, 'e': 1}
4
Isolated vertices:
None
Graph density:
0.42857142857142855
0.0
1.0
Degree sequence:
(4, 4, 3, 2, 2, 2, 1)
Erdos GAllai:
True
False
Clustering:
0.5
1.0


In [23]:
graph_2comps = Graph(G)
graph_2comps.add_vertex('i')
graph_2comps.add_vertex('j')
graph_2comps.add_vertex('k')
graph_2comps.add_vertex('l')
graph_2comps.add_vertex('m')
graph_2comps.add_edge({'i','j'})
graph_2comps.add_edge({'j','k'})
graph_2comps.add_edge({'k','l'})
graph_2comps.add_edge({'l','m'})
print(len(set(graph_2comps.connected_components().values())))

2


In [24]:
print(graph_2comps.shortest_path('a','i'))
print(graph_2comps.shortest_path('a','b'))
print(graph_2comps.shortest_path('e','b'))
print(graph_2comps.shortest_path('i','m'))

inf
2
3
4


In [25]:
print(graph_2comps.diameter_component('a'))
print(graph_2comps.diameter_component('i'))
print(graph_2comps.diameter())
print(graph_2comps.biggest_component_diameter())

3
4
inf
3


In [28]:
zachary = Graph.from_txt('zachary_connected.txt')
random100 = Graph.from_txt('graph_100n_1000m.txt')
random1000 = Graph.from_txt('graph_1000n_4000m.txt')

In [27]:
Latex(r"""\begin{array}{|c|c|c|c|c|c|} \hline \text{Dataset} & \text{Number of vertices} & \text{Number of edges} & 
\text{Density} & \text{Diameter} & \text{Clustering coefficient} \\ \hline 
\text{Zachary} & """
+ str(len(zachary.vertices())) + r' & '+ str(len(zachary.edges())) + r""" & """
+ str(zachary.density()) + """ & """ + str(zachary.diameter()) + """ & """ + str(zachary.global_clustering_coefficient()) 
+ r""" \\\ \text{Random, } N=10^2 & """
+ str(len(random100.vertices())) + """ & """ + str(len(random100.edges())) + """ & """
+ str(random100.density()) + """ & """ + str(random100.diameter()) + """ & """ + str(random100.global_clustering_coefficient())
+ r""" \\\ \text{Random, }N=10^3 & """
+ str(len(random1000.vertices())) + """ & """ + str(len(random1000.edges())) + """ & """
+ str(random1000.density()) + """ & """ + str(random1000.diameter()) + """ & """ + str(random1000.global_clustering_coefficient()) 
+ """\\\ \hline \end{array}""")

<IPython.core.display.Latex object>